# Mock Lambda Function

In [1]:
test_comment = "The unquestioned value of crypto currencies is that they provide an easy way for organized crime syndicates of all stripes to launder their ill gotten millions and billions while making suckers out of governments and the rest of us."

test_event = {'comment': test_comment}
test_context = None

In [2]:
import boto3

s3 = boto3.resource('s3')
obj = s3.Object('sagemaker-us-east-1-228889150161','data/nyt-features/vocab.json')
obj.download_file('vocab.json')

In [3]:
import os 
os.environ['ENDPOINT'] = 'ntm-nyt'

In [4]:
import json 
import re 
from collections import Counter
import boto3
import io
import os


def load_vocab():
    with open('vocab.json', 'r') as json_file:
        return json.load(json_file)

def mock_function(event, context):
    vocab = load_vocab()
    tokens = re.split(r'\W+', event['comment'])
    
    lowercase = map(lambda token: token.lower(), tokens)
    vocab_words = filter(lambda token: token in vocab, lowercase)
    idx = map(lambda token: vocab[token], vocab_words)
    counts = Counter(idx)
    
    #https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html
    keys = sorted(counts.keys())
    values = list(map(lambda key: counts[key], keys))
    features = { "keys": keys, "shape": [len(vocab)],"values": values }
    
    ntm_input = {"instances":[{"data": {"features": features}}]}
    req_body = json.dumps(ntm_input)
    print(req_body)
    
    client = boto3.client('sagemaker-runtime')
    resp = client.invoke_endpoint(
        EndpointName=os.environ['ENDPOINT'],
        Body=req_body.encode('utf-8'),
        ContentType='application/json',
        Accept='application/json'
    )
        
    predictions = json.load(resp["Body"])
    return predictions

In [ ]:
response = mock_function(test_event, test_context)

In [ ]:
print(response)

In [ ]:
import sagemaker
sagemaker_session = sagemaker.Session()
sagemaker_session.delete_endpoint('ntm-nyt')